In [29]:
category_alias = {}
def categorize(category, inverse = 0):
    if not inverse:
        if category in category_alias:
            return category_alias[category]
        if category_alias:
            category_alias[category] = max(category_alias.values()) + 1
        else:
            category_alias[category] = 0
        return category_alias[category]
    else:
        for translation, alias in category_alias.items():
            if alias == category:
                return translation
        return "None"

def training(input_file, model='CBOW'):  
    ## Load a model to train
    if model == 'CBOW':
        from gensim.models import Word2Vec
        #model = Word2Vec(sentences=list(corpus.values()), sg = 0)
        #model.save("../data/word2vec-cbow.model")
        
    elif model == 'SG':
        from gensim.models import Word2Vec
        #model = Word2Vec(sentences=list(corpus.values()), sg = 1)
        #model.save("../data/word2vec-sg.model")
    elif model == "BAYES":
        import pandas as pd 
        df = pd.read_csv(input_file, delimiter='\t')
       
        ## Convertimos cada genero a un valor numérico
        df['genero'] = df['genero'].apply( lambda c : categorize(c) )
        
        #print(f"{df}")
        
        ## Entrenamiento, creando vectores por las palabras
        from sklearn.model_selection import train_test_split
        films_train, films_test, response_train, response_test = train_test_split( df.sinopsis, df.genero, test_size = .25)
        
        #print(f"{films_train}, {response_train}")
        
        from sklearn.feature_extraction.text import CountVectorizer
        vector = CountVectorizer()
        films_train_count = vector.fit_transform(films_train)
        
        #print(f"{films_train_count}")
        
        from sklearn.naive_bayes import MultinomialNB
        model = MultinomialNB()
        model.fit(films_train_count, response_train)
        
        return (model,vector)
    else:
        print(f"ERROR unknown model {model}")

In [25]:
def predict(synopsis, model, vector, type_model = 'CBOW'):
    if type_model == 'CBOW':
        pass
    elif type_model == 'SG':
        pass
    elif type_model == "BAYES":
        if type(synopsis) is str:
            synopsis = [synopsis]
        synopsis = vector.transform(synopsis)
        return model.predict(synopsis)
    else:
        print(f"ERROR unknown model {type_model}")

In [30]:
input_file = '../data/stemming_data.txt'
(model,vector) = training(input_file, "BAYES")

predictions = predict('Las Aes Sadai, una poderosa fortaleza de mujeres, parecen dominar la magia por su capacidad de contactar con el Poder Único que se obtiene de la Fuente Verdadera, que hace girar la vital Rueda del Tiempo. La Época de la locura ha llegado por la contaminación de una parte de la fuente dejando un mundo arruinado y desorganizado en su forma de vida', model, vector, 'BAYES')

for category in predictions:
    print(f"Film has a category of {categorize(category, 1)}")

Film has a category of Comedia
